In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import scipy.stats as stats
from sklearn import metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AR DATASET/merged_processed_data.csv')
data

,Unnamed: 0,Unnamed: 0.1,user_id,datetime,x,y,z,label
0,0,345818,1,2018-07-25 11:14:18.104000+10:00,-0.306,0.612,9.779,5.0
1,1,67171,1,2018-07-25 17:25:32.409000+10:00,2.221,9.538,0.116,8.0
2,2,67154,1,2018-07-25 17:25:32.409000+10:00,2.221,9.538,0.116,8.0
3,3,103187,1,2018-07-25 17:25:32.208000+10:00,2.145,9.385,0.158,8.0
4,4,103230,1,2018-07-25 17:25:32.208000+10:00,2.145,9.385,0.158,8.0
...,...,...,...,...,...,...,...,...
1908851,1908851,4419939,51,2018-12-24 10:46:07.573000+09:00,-0.612,9.653,0.005,12.0
1908852,1908852,4419940,51,2018-12-24 10:46:07.573000+09:00,-0.612,9.653,0.005,12.0
1908853,1908853,4419937,51,2018-12-24 10:46:07.567000+09:00,-0.612,9.653,0.005,12.0
1908854,1908854,4419936,51,2018-12-24 10:46:05.757000+09:00,3.524,9.998,0.227,12.0


In [ ]:
data['user_id'].value_counts()

18    955258
4     444776
7     184193
1     139564
19    131225
8      27407
51     25587
38       846
Name: user_id, dtype: int64

In [ ]:
data['label'].value_counts()

5.0     1077203
4.0      301476
7.0      118950
12.0     110005
3.0      109230
8.0       55437
9.0       48034
1.0       37984
10.0      13781
2.0       13554
11.0      12822
6.0       10380
Name: label, dtype: int64

# Balancing

In [ ]:
# cnt = data['label'].value_counts()
# mn = min(cnt);
# print(mn)

In [ ]:
# labels = len(cnt) 
# balanced_data = pd.DataFrame()
# for i in range(1,labels+1):
#   balanced_data = balanced_data.append(data[data['label']==i].head(mn).copy())
# balanced_data.shape

In [ ]:
# balanced_data['label'].value_counts()

**Standardization**

In [ ]:
X = data[['x','y','z']]
y = data['label']

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

scaled_X = pd.DataFrame(data=X, columns=['x', 'y', 'z'])
scaled_X['label'] = y.values

scaled_X

,x,y,z,label
0,-0.146030,-0.819383,1.881102,5.0
1,0.297393,1.181110,-0.539026,8.0
2,0.297393,1.181110,-0.539026,8.0
3,0.284057,1.146820,-0.528507,8.0
4,0.284057,1.146820,-0.528507,8.0
...,...,...,...,...
1908851,-0.199725,1.206884,-0.566826,12.0
1908852,-0.199725,1.206884,-0.566826,12.0
1908853,-0.199725,1.206884,-0.566826,12.0
1908854,0.526036,1.284205,-0.511226,12.0


**Framing**

In [ ]:
frame_size = 240
hop_size = 15

In [ ]:
def get_frames(df, frame_size, hop_size):
    
    N_FEATURES = 3
    
    frames = []
    labels = []
    
    for i in range(0, len(df)-frame_size, hop_size):
        x = df['x'].values[i: i+frame_size]
        y = df['y'].values[i: i+frame_size]
        z = df['z'].values[i: i+frame_size]
        
        label = stats.mode(df['label'][i: i+frame_size])[0][0]

        frames.append([x, y, z])
        labels.append(label)
        
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [ ]:
Frame_X, y = get_frames(scaled_X, frame_size, hop_size)
Frame_X.shape, y.shape

((127242, 240, 3), (127242,))

**Feature Extraction (Mean, Median, Mode, Variance, Standard Deviation, RMS)**

In [ ]:
X = []
# Mean_X = []
# Median_X = []
# Mode_X = []
# Var_X = []
# Std_X = []
# RMS_X = []
N_FEATURES = 6
for i in range(Frame_X.shape[0]):
  mean_X = np.mean(Frame_X[i],axis=0)
  median_X = np.median(Frame_X[i],axis=0)
  mode_X = np.asarray(stats.mode(Frame_X[i])[0][0])
  variance_X = np.var(Frame_X[i],axis=0)
  std_deviation_X = np.std(Frame_X[i],axis=0)
  rms_X = np.sqrt(np.mean(np.square(Frame_X[i]),axis=0))
  # comb = np.array([mean_X,median_X,mode_X,variance_X,std_deviation_X,rms_X])
  comb = np.array([mean_X,median_X,mode_X,variance_X,std_deviation_X,rms_X])
  X.append(comb.reshape(-1))
  # Mean_X.append(mean_X)
  # Median_X.append(median_X)
  # Mode_X.append(mode_X)
  # Var_X.append(variance_X)
  # Std_X.append(std_deviation_X)
  # RMS_X.append(rms_X)
X = np.asarray(X)
# Mean_X = np.asarray(Mean_X)
# Median_X = np.asarray(Median_X)
# Mode_X = np.asarray(Mode_X)
# Var_X = np.asarray(Var_X)
# Std_X = np.asarray(Std_X)
# RMS_X = np.asarray(RMS_X)
X.shape

(127242, 18)

**Training and Testing data split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((101793, 18), (25449, 18), (101793,), (25449,))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

rf.fit(X_train, y_train);

predictions = rf.predict(X_test)

errors = abs (predictions - y_test)
print('Mean Absolute Error: ', round(np.mean(errors),2))

Mean Absolute Error:  0.58


# Performance

In [ ]:
#Mean Absolute Percentage Error
mape = 100*(errors/y_test) 

In [ ]:
accuracy = 100 - np.mean(mape)
print('Accuracy: ', round(accuracy,2), '%')

Accuracy:  85.04 %


In [ ]:
#Score
print('Score: ',rf.score(X_test, y_test)*100,'%')

Score:  70.16656448452298 %


In [ ]:
y_pred = np.round(predictions) 
y_pred

array([5., 8., 8., ..., 5., 5., 7.])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[   65,   104,   111,   140,    88,    24,     4,     4,     0,
            0,     0,     0],
       [    1,    39,    38,    39,    30,     9,     5,     5,     0,
            0,     0,     0],
       [    0,     0,   213,   658,   411,   111,    19,     4,     5,
            4,     0,     0],
       [    0,     1,     5,  1993,  1568,   303,    70,    23,    10,
            5,     2,     0],
       [    0,     0,     9,   169, 13334,   761,   110,    28,     7,
            2,     1,     0],
       [    0,     0,     0,     1,    24,    72,    17,     2,     2,
            1,     0,     0],
       [    0,     0,     2,    11,   288,   663,   577,    26,     7,
            2,     0,     0],
       [    0,     0,     2,     5,    36,   106,   143,   332,   121,
           20,     5,     0],
       [    0,     0,     0,     4,    16,    58,   100,   136,   311,
           21,     1,     0],
       [    0,     0,     0,     0,     4,     8,    13,    27,    35,
           68,     6

processdata lab -

(unbalanced) (30-10) -> 51.61

(balanced) (30-10) -> 71.15



In [ ]:
# frame_size = 240
# hop_size = 30
#Score:  73.10925438281703 %
#with balanced user datqaa

In [ ]:
# frame_size = 240
# hop_size = 15
#Score:  87.95500213926748 %
#with balanced user data only

In [ ]:
# frame_size = 240
# hop_size = 15
#Score:  76.8757029755651 %
#with balanced merged data